In [13]:
import os
os.environ['SPARK_HOME'] = "/Users/dimitr/Desktop/data_engineer_courses/lesson_06/Spark"
os.environ['PYSPARK_DRIVER_PYTHON'] = 'jupyter'
os.environ['PYSPARK_DRIVER_PYTHON_OPTS'] = 'lab'
os.environ['PYSPARK_PYTHON'] = 'python'

In [14]:
from pyspark.sql import SparkSession

In [84]:
from pyspark.sql.functions import count, desc, sum

In [26]:
import pandas as pd

In [16]:
spark = SparkSession.builder.appName('PySpark-Get-Started').getOrCreate()

In [21]:
spark.version

'3.5.0'

In [23]:
actor_df = spark.read.csv('./data/actor.csv', header=True, inferSchema=True)
address_df = spark.read.csv('./data/address.csv', header=True, inferSchema=True)
category_df = spark.read.csv('./data/category.csv', header=True, inferSchema=True)
city_df = spark.read.csv('./data/city.csv', header=True, inferSchema=True)
country_df = spark.read.csv('./data/country.csv', header=True, inferSchema=True)
customer_df = spark.read.csv('./data/customer.csv', header=True, inferSchema=True)
film_df = spark.read.csv('./data/film.csv', header=True, inferSchema=True)
film_actor_df = spark.read.csv('./data/film_actor.csv', header=True, inferSchema=True)
film_category_df = spark.read.csv('./data/film_category.csv', header=True, inferSchema=True)
inventory_df = spark.read.csv('./data/inventory.csv', header=True, inferSchema=True)
language_df = spark.read.csv('./data/language.csv', header=True, inferSchema=True)
payment_df = spark.read.csv('./data/payment.csv', header=True, inferSchema=True)
rental_df = spark.read.csv('./data/rental.csv', header=True, inferSchema=True)
staff_df = spark.read.csv('./data/staff.csv', header=True, inferSchema=True)
store_df = spark.read.csv('./data/store.csv', header=True, inferSchema=True)

In [83]:
film_category_df.show()

+-------+-----------+-------------------+
|film_id|category_id|        last_update|
+-------+-----------+-------------------+
|      1|          6|2022-02-15 11:07:09|
|      2|         11|2022-02-15 11:07:09|
|      3|          6|2022-02-15 11:07:09|
|      4|         11|2022-02-15 11:07:09|
|      5|          8|2022-02-15 11:07:09|
|      6|          9|2022-02-15 11:07:09|
|      7|          5|2022-02-15 11:07:09|
|      8|         11|2022-02-15 11:07:09|
|      9|         11|2022-02-15 11:07:09|
|     10|         15|2022-02-15 11:07:09|
|     11|          9|2022-02-15 11:07:09|
|     12|         12|2022-02-15 11:07:09|
|     13|         11|2022-02-15 11:07:09|
|     14|          4|2022-02-15 11:07:09|
|     15|          9|2022-02-15 11:07:09|
|     16|          9|2022-02-15 11:07:09|
|     17|         12|2022-02-15 11:07:09|
|     18|          2|2022-02-15 11:07:09|
|     19|          1|2022-02-15 11:07:09|
|     20|         12|2022-02-15 11:07:09|
+-------+-----------+-------------

In [51]:
# 1. Вивести кількість фільмів в кожній категорії. Результат відсортувати за спаданням.
category_counts = film_category_df.groupBy('category_id').agg(count("*").alias("movie_count"))
sorted_category_counts = category_counts.orderBy(desc("movie_count"))
sorted_category_counts.show()

+-----------+-----------+
|category_id|movie_count|
+-----------+-----------+
|         15|         74|
|          9|         73|
|          8|         69|
|          6|         68|
|          2|         66|
|          1|         64|
|         13|         63|
|          7|         62|
|         10|         61|
|         14|         61|
|          3|         60|
|          5|         58|
|         16|         57|
|          4|         57|
|         11|         56|
|         12|         51|
+-----------+-----------+



In [56]:
# 2. Вивести 10 акторів, чиї фільми брали на прокат найбільше. Результат відсортувати за спаданням.
actor_counts = film_actor_df.groupBy("actor_id").agg(count("*").alias("film_count"))
sorted_actor_counts = actor_counts.orderBy(desc("film_count"))
top_10_actors = sorted_actor_counts.limit(10)
top_10_actors.show()

+--------+----------+
|actor_id|film_count|
+--------+----------+
|     107|        42|
|     102|        41|
|     198|        40|
|     181|        39|
|      23|        37|
|      81|        36|
|      13|        35|
|     106|        35|
|      37|        35|
|      60|        35|
+--------+----------+



In [88]:
# 3. Вивести категорія фільмів, на яку було витрачено найбільше грошей в прокаті
joined_df = film_df.join(film_category_df, 'film_id')
category_costs = joined_df.groupBy("category_id").agg(sum("replacement_cost").alias("total_cost"))
most_expensive_category = category_costs.orderBy(desc("total_cost"))
most_expensive_category.show()
#most_expensive_category = category_costs.orderBy(desc("total_cost")).first()
#most_expensive_category.show()

+-----------+------------------+
|category_id|        total_cost|
+-----------+------------------+
|         15|1509.2600000000004|
|          8|1361.3100000000004|
|          9|1361.2700000000004|
|          1|1338.3600000000004|
|          6|1334.3200000000004|
|          2|1328.3400000000004|
|          7|1307.3800000000003|
|         14|1290.3900000000003|
|         10|1237.3900000000003|
|         13|1223.3700000000003|
|          3|1203.4000000000003|
|          4|1197.4300000000003|
|         11|1112.4400000000003|
|          5|1103.4200000000003|
|         16|1084.4300000000003|
|         12| 991.4900000000004|
+-----------+------------------+



In [67]:
# 4. Вивести назви фільмів, яких не має в inventory.
missing_movies = film_df.join(inventory_df, film_df["film_id"] == inventory_df["film_id"], "left_anti")
missing_movies.select("title").show(truncate=False)

+----------------------+
|title                 |
+----------------------+
|ALICE FANTASIA        |
|APOLLO TEEN           |
|ARGONAUTS TOWN        |
|ARK RIDGEMONT         |
|ARSENIC INDEPENDENCE  |
|BOONDOCK BALLROOM     |
|BUTCH PANTHER         |
|CATCH AMISTAD         |
|CHINATOWN GLADIATOR   |
|CHOCOLATE DUCK        |
|COMMANDMENTS EXPRESS  |
|CROSSING DIVORCE      |
|CROWDS TELEMARK       |
|CRYSTAL BREAKING      |
|DAZED PUNK            |
|DELIVERANCE MULHOLLAND|
|FIREHOUSE VIETNAM     |
|FLOATS GARDEN         |
|FRANKENSTEIN STRANGER |
|GLADIATOR WESTWARD    |
+----------------------+
only showing top 20 rows



In [77]:
# 5. Вивести топ 3 актори, які найбільше зʼявлялись в категорії фільмів “Children”
category_name = "Children"

children_films = category_df.filter(category_df.name == category_name) \
    .join(film_category_df, category_df.category_id == film_category_df.category_id) \
    .select(film_category_df.film_id)

top_actors = children_films \
    .join(film_actor_df, children_films.film_id == film_actor_df.film_id) \
    .groupBy("actor_id") \
    .agg(count("*").alias("appearance_count"))

number_top_actors = 3
top_actors \
    .orderBy(desc("appearance_count")) \
    .show(number_top_actors)

+--------+----------------+
|actor_id|appearance_count|
+--------+----------------+
|      17|               7|
|      66|               5|
|     140|               5|
+--------+----------------+
only showing top 3 rows



In [89]:
spark.stop()